In [1]:
from os import listdir
import pandas as pd

In [2]:
# listando os arquivos com dados metereologicos de todas as estacoes
# fonte: https://bdmep.inmet.gov.br/#
arquivos_em = listdir('dados/metereologicos/')


In [3]:
dados_climaticos = pd.DataFrame()

for index, arquivo in enumerate(arquivos_em):   
    # criando uma string com o nome do número da estação metereologica
    estacao = arquivo[:11]; 
    
    # atribuindo a string a um nome de variável
    globals()[estacao] = pd.read_csv('dados/metereologicos/' + arquivo, sep=";", skiprows=10)
    
    # obtendo o nome da estacao na primeira linha do aquivo
    with open('dados/metereologicos/' + arquivo, 'r') as arquivo:
        for texto in arquivo.readlines():

            if (texto.startswith('Nome')):
                nome_estacao = texto[6:]

            if (texto.startswith('Codigo')):
                codigo_estacao = texto[16:]
                        
    # adicionando uma coluna com o nome da estacao    
    globals()[estacao].insert(1, 'DC_NOME', nome_estacao.replace('\n',''))
    globals()[estacao].insert(2, 'CD_ESTACAO', codigo_estacao.replace('\n',''))
    
    # consolidando todos os dados em um único dataframe
    dados_climaticos = pd.concat([dados_climaticos,globals()[estacao]])
    

    
    
    

In [4]:
# removendo colunas não utilizadas, e excluindo dados nulos
dados_climaticos.drop(['Unnamed: 1', 'Unnamed: 7'], axis=1, inplace=True)
dados_climaticos.dropna()



,Data Medicao,DC_NOME,CD_ESTACAO,"NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(número)","PRECIPITACAO TOTAL, MENSAL(mm)","TEMPERATURA MAXIMA MEDIA, MENSAL(°C)","TEMPERATURA MEDIA COMPENSADA, MENSAL(°C)","TEMPERATURA MINIMA MEDIA, MENSAL(°C)","UMIDADE RELATIVA DO AR, MEDIA MENSAL(%)"
0,2008-01-31,ARACUAI,83442,9.0,47.4,33.887097,26.696129,21.664516,63.540323
1,2008-02-29,ARACUAI,83442,13.0,136.6,34.151724,26.688276,21.317241,67.051724
2,2008-03-31,ARACUAI,83442,11.0,84.7,33.070968,26.236129,21.322581,68.830645
3,2008-04-30,ARACUAI,83442,3.0,54.1,33.796667,26.230667,20.756667,67.333333
4,2008-05-31,ARACUAI,83442,0.0,0.0,31.554839,23.929677,17.745161,58.733871
...,...,...,...,...,...,...,...,...,...
175,2022-08-31,BELEM,82191,13.0,103.9,34.303226,27.372258,22.696774,79.685484
176,2022-09-30,BELEM,82191,16.0,152.3,34.546667,27.581333,23.043333,77.866667
177,2022-10-31,BELEM,82191,18.0,139.8,34.312903,27.720000,23.203226,78.927419
178,2022-11-30,BELEM,82191,19.0,359.2,33.300000,27.302000,23.653333,83.283333


In [5]:
# Obtendo informações sobre as estacoes metereologicas:
# https://portal.inmet.gov.br/

estacoes_convencionais = pd.read_csv('dados/estacoes/CatalogoEstacoesAutomaticas.csv', sep=";")
estacoes_automaticas = pd.read_csv('dados/estacoes/CatalogoEstacoesConvencionais.csv', sep=";")

In [6]:
# criando coluna para identificar o tipo da estacao
estacoes_convencionais['TIPO'] = 'convencional'
estacoes_automaticas['TIPO'] = 'automatica'

In [7]:
# unindo os dados de todas as estacoes em um único dataframe
estacoes = pd.concat([estacoes_automaticas, estacoes_convencionais])

In [8]:
# filtrando somente as estacoes operantes
estacoes_operantes = estacoes.query('CD_SITUACAO == "Operante"')

In [9]:
# filtrando somente as colunas necessárias
estacoes_operantes = estacoes_operantes[['DC_NOME', 'SG_ESTADO', 'CD_ESTACAO', 'TIPO']]

In [10]:
estacoes_operantes.head()

,DC_NOME,SG_ESTADO,CD_ESTACAO,TIPO
0,AGUA BRANCA,AL,82989,automatica
1,ALTAMIRA,PA,82353,automatica
2,ALTO DA BOA VISTA,RJ,83007,automatica
3,APODI,RN,82590,automatica
4,ARACUAI,MG,83442,automatica


In [11]:
dados_climaticos['CD_ESTACAO'] = dados_climaticos['CD_ESTACAO'].astype('int64')

In [15]:
# fazendo um merge entre as estacoes operantes e os dados metereologicos
dados_consolidados = estacoes_operantes.merge(dados_climaticos,  how='inner', on='CD_ESTACAO')

In [16]:
dados_consolidados.dropna(inplace=True)

In [17]:
dados_consolidados.drop('DC_NOME_y', axis=1, inplace=True)
dados_consolidados.rename(columns={"DC_NOME_x":"DC_NOME"}) 

,DC_NOME,SG_ESTADO,CD_ESTACAO,TIPO,Data Medicao,"NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(número)","PRECIPITACAO TOTAL, MENSAL(mm)","TEMPERATURA MAXIMA MEDIA, MENSAL(°C)","TEMPERATURA MEDIA COMPENSADA, MENSAL(°C)","TEMPERATURA MINIMA MEDIA, MENSAL(°C)","UMIDADE RELATIVA DO AR, MEDIA MENSAL(%)"
0,AGUA BRANCA,AL,82989,automatica,2008-01-31,1.0,0.2,31.583871,24.574194,18.854839,73.806452
2,AGUA BRANCA,AL,82989,automatica,2008-03-31,14.0,192.0,28.935484,23.780000,19.261290,87.600000
3,AGUA BRANCA,AL,82989,automatica,2008-04-30,11.0,138.6,28.073333,23.429333,19.230000,90.766667
4,AGUA BRANCA,AL,82989,automatica,2008-05-31,22.0,142.4,25.670968,21.641290,18.348387,93.700000
5,AGUA BRANCA,AL,82989,automatica,2008-06-30,20.0,93.2,24.230000,20.217333,16.833333,93.900000
...,...,...,...,...,...,...,...,...,...,...,...
27355,ZE DOCA,MA,82376,automatica,2022-08-31,4.0,8.1,33.945161,28.003226,23.280645,70.403226
27356,ZE DOCA,MA,82376,automatica,2022-09-30,2.0,16.6,34.790000,28.394483,23.700000,67.333333
27357,ZE DOCA,MA,82376,automatica,2022-10-31,4.0,49.3,35.373077,28.800000,23.996154,71.038462
27358,ZE DOCA,MA,82376,automatica,2022-11-30,11.0,147.1,33.826923,27.892800,23.750000,79.115385


In [19]:
dados_consolidados.columns

Index(['DC_NOME_x', 'SG_ESTADO', 'CD_ESTACAO', 'TIPO', 'Data Medicao',
       'NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(número)',
       'PRECIPITACAO TOTAL, MENSAL(mm)',
       'TEMPERATURA MAXIMA MEDIA, MENSAL(°C)',
       'TEMPERATURA MEDIA COMPENSADA, MENSAL(°C)',
       'TEMPERATURA MINIMA MEDIA, MENSAL(°C)',
       'UMIDADE RELATIVA DO AR, MEDIA MENSAL(%)'],
      dtype='object')

In [20]:
dados_consolidados.columns = ['NM_ESTACAO', 'SG_UF_ESTADO', 'CD_ESTACAO', 'TP_ESTACAO', 'DT_MEDICAO',
       'NUMERO_DIAS_COM_PRECIP_PLUV',
       'PRECIPITACAO_TOTAL_EM_MILIMETROS',
       'TEMPERATURA_MAXIMA_MEDIA',
       'TEMPERATURA_MEDIA_COMPENSADA',
       'TEMPERATURA_MINIMA_MEDIA',
       'UMIDADE_RELATIVA_DO_AR_MEDIA MENSAL(%)']

In [21]:
dados_consolidados

,NM_ESTACAO,SG_UF_ESTADO,CD_ESTACAO,TP_ESTACAO,DT_MEDICAO,NUMERO_DIAS_COM_PRECIP_PLUV,PRECIPITACAO_TOTAL_EM_MILIMETROS,TEMPERATURA_MAXIMA_MEDIA,TEMPERATURA_MEDIA_COMPENSADA,TEMPERATURA_MINIMA_MEDIA,UMIDADE_RELATIVA_DO_AR_MEDIA MENSAL(%)
0,AGUA BRANCA,AL,82989,automatica,2008-01-31,1.0,0.2,31.583871,24.574194,18.854839,73.806452
2,AGUA BRANCA,AL,82989,automatica,2008-03-31,14.0,192.0,28.935484,23.780000,19.261290,87.600000
3,AGUA BRANCA,AL,82989,automatica,2008-04-30,11.0,138.6,28.073333,23.429333,19.230000,90.766667
4,AGUA BRANCA,AL,82989,automatica,2008-05-31,22.0,142.4,25.670968,21.641290,18.348387,93.700000
5,AGUA BRANCA,AL,82989,automatica,2008-06-30,20.0,93.2,24.230000,20.217333,16.833333,93.900000
...,...,...,...,...,...,...,...,...,...,...,...
27355,ZE DOCA,MA,82376,automatica,2022-08-31,4.0,8.1,33.945161,28.003226,23.280645,70.403226
27356,ZE DOCA,MA,82376,automatica,2022-09-30,2.0,16.6,34.790000,28.394483,23.700000,67.333333
27357,ZE DOCA,MA,82376,automatica,2022-10-31,4.0,49.3,35.373077,28.800000,23.996154,71.038462
27358,ZE DOCA,MA,82376,automatica,2022-11-30,11.0,147.1,33.826923,27.892800,23.750000,79.115385
